**Machine Learning Operations -- Assignment 2**

**11/1/2023**

**Danil Meresenschi, Jason Rajan, Ian Vogt, Monicha Zhang**

**Create Handle to Workspace**

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()
# # Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="5405ca59-d0a7-4e18-af16-b7f21ebcb1b0",
    resource_group_name="MLOps_group6",
    workspace_name="MLOps_Assignment2",
)
cpu_cluster = None

**Access Registered Data Asset**

In [2]:
import mltable
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#ml_client = MLClient.from_config(credential=DefaultAzureCredential())
#data_asset = ml_client.data.get("Assignment2_test_data", version="1")

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("Assignment2_test_data", version="1")

print(data_asset.path)
path = {
  'folder': data_asset.path
}
df = pd.read_csv(data_asset.path)
#tbl = mltable.from_delimited_files(paths=[path])
#df = tbl.to_pandas_dataframe()
df

Found the config file in: /config.json


azureml://subscriptions/5405ca59-d0a7-4e18-af16-b7f21ebcb1b0/resourcegroups/mlops_group6/workspaces/mlops_assignment2/datastores/workspaceblobstore/paths/UI/2023-10-31_025318_UTC/athletes.csv/


,athlete_id,name,region,team,affiliate,gender,age,height,weight,fran,...,snatch,deadlift,backsq,pullups,eat,train,background,experience,schedule,howlong
0,2554.0,Pj Ablang,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,...,NaN,400.0,305.0,NaN,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
1,3517.0,Derek Abdella,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,...,NaN,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5164.0,Abo Brandon,Southern California,LAX CrossFit,LAX CrossFit,Male,40.0,67.0,NaN,211.0,...,200.0,375.0,325.0,25.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|,4+ years|
4,5286.0,Bryce Abbey,NaN,NaN,NaN,Male,32.0,65.0,149.0,206.0,...,150.0,NaN,325.0,50.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I inc...,I played college sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I strictly s...,1-2 years|
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423001,574489.0,Odo Renata,Latin America,Team Guarujá Inox,CrossFit Guaruja,Female,36.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423002,585696.0,Lozzie Trevor,Australia,FBP CrossFit Games Team,FBP CrossFit,Female,27.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423003,608828.0,Marisol Smith,North West,CrossFit Oak Harbor,CrossFit Oak Harbor,Female,44.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423004,628881.0,Pedrini Morgane,Europe,NaN,CrossFit 67,Female,20.0,64.0,61.0,NaN,...,NaN,80.0,143.0,NaN,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|,NaN,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I strictly s...,6-12 months|


**Create a Compute Resource**

In [3]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure Machine Learning compute object with the intended parameters
    # if you run into an out of quota error, change the size to a comparable VM that is available.
    # Learn more on https://azure.microsoft.com/en-us/pricing/details/machine-learning/.
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure Machine Learning Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS3_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )
    print(
        f"AMLCompute with name {cpu_cluster.name} will be created, with compute size {cpu_cluster.size}"
    )
    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

You already have a cluster named cpu-cluster, we'll reuse it as is.


**Environment and Dependencies**

In [4]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [5]:
# Can be updated as we go
#%%writefile {dependencies_dir}/conda.yaml
#name: model-env
#channels:
#  - conda-forge
#dependencies:
#  - python=3.8
#  - numpy=1.21.2
#  - pip=21.2.4
#  - scikit-learn=0.24.2
#  - scipy=1.7.1
#  - 'pandas>=1.1,<1.2'
#  - pip:
#    - inference-schema[numpy-support]==1.3.0
#    - xlrd==2.0.1
#    - mlflow==2.4.1
#    - azureml-mlflow==1.51.0
#    - codecarbon==2.3.1

In [6]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-scikit-learn"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for MLOps Assignment 2 pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="0.2.0",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name aml-scikit-learn is registered to workspace, the environment version is 0.2.0


**Create Data Prep Component**

In [7]:
import os

data_prep_src_dir = "./components/data_prep"
os.makedirs(data_prep_src_dir, exist_ok=True)

In [8]:
%%writefile {data_prep_src_dir}/data_prep.py
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import logging
import mlflow

def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.2)
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    args = parser.parse_args()

    # Start Logging
    mlflow.start_run()

    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("input data:", args.data)
    
    #df = pd.read_csv(args.data, header=1, index_col=0, engine='python', encoding='utf-8')
    df = pd.read_csv(args.data)
    # Retain only the first 423006 rows and 27 columns (original dataset)
    #data = df.iloc[:423006, :27]
    data = df
    # Remove not relevant columns
    data = data.dropna(subset=['region','age','weight','height','howlong','gender','eat', \
                           'train','background','experience','schedule', \
                           'deadlift','candj','snatch','backsq'])
    data = data.drop(columns=['affiliate','team','name','athlete_id','fran','helen','grace',\
                            'filthy50','fgonebad','run400','run5k','pullups','train'])

    data = data[data['weight'] < 1500]
    data = data[data['gender'] != '--']
    data = data[data['age'] >= 18]
    data = data[(data['height'] < 96) & (data['height'] > 48)]

    data = data[(data['deadlift'] > 0) & (data['deadlift'] <= 1105)|((data['gender'] == 'Female') \
                & (data['deadlift'] <= 636))]

    data = data[(data['candj'] > 0) & (data['candj'] <= 395)]
    data = data[(data['snatch'] > 0) & (data['snatch'] <= 496)]
    data = data[(data['backsq'] > 0) & (data['backsq'] <= 1069)]

    #Dummy encode eat column
    unique_outcomes = set()
    for response in data['eat']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"eat_{outcome}"
        data[column_name] = data['eat'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'eat' column if needed
    data = data.drop(columns=['eat','eat_'])

    #binary encode background column
    unique_outcomes = set()
    for response in data['background']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"background_{outcome}"
        data[column_name] = data['background'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'background' column if needed
    data = data.drop(columns=['background','background_'])

    #binary encode experience column
    unique_outcomes = set()
    for response in data['experience']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"experience_{outcome}"
        data[column_name] = data['experience'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'background' column if needed
    data = data.drop(columns=['experience','experience_'])

    #binary encode schedule column
    unique_outcomes = set()
    for response in data['schedule']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"schedule_{outcome}"
        data[column_name] = data['schedule'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'background' column if needed
    data = data.drop(columns=['schedule','schedule_'])

    #binary encode howlong column
    unique_outcomes = set()
    for response in data['howlong']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"howlong_{outcome}"
        data[column_name] = data['howlong'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'background' column if needed
    data = data.drop(columns=['howlong','howlong_'])

    # Creating a region dictionary for maping
    region_mapping = {
        # Region 1 Group
        'Africa': 'Region_1',
        'Asia': 'Region_1',
        'Australia': 'Region_1',
        'Latin America': 'Region_1',
        
        # Region 2 Group
        'Canada East': 'Region_2',
        'Canada West': 'Region_2',

        # Region 3 Group
        'Northern California': 'Region_3',
        'Southern California': 'Region_3',
        
        # Region 4 Group
        'Central East': 'Region_4',

        # Region 5 Group
        'Mid Atlantic': 'Region_5',

        # Region 6 Group
        'North Central': 'Region_6',
        'South Central': 'Region_6',

        # Region 7 Group
        'North East': 'Region_7',
        'North West': 'Region_7',

        # Region 8 Group  
        'South East': 'Region_8',
        'South West': 'Region_8',
        
        # Region 9 Group
        'Europe': 'Region_9'
    }

    # Apply the mapping
    data['region'] = data['region'].map(region_mapping)

    #encode region and gender
    le=LabelEncoder()
    data["le_region"] = le.fit_transform(data['region'])
    data["le_gender"] = le.fit_transform(data['gender'])

    #Lets get the crosswalk for region and gender
    data.groupby(['region', 'le_region']).size().reset_index(name='counts')

    data.groupby(['gender', 'le_gender']).size().reset_index(name='counts')

    #drop unencoded region/gender columns
    data = data.drop(columns=['region','gender'])
    #calculate our total_lift target value using clean and jerk, snatch,deadlift and backsquat values
    data['total_lift'] = data['candj'] + data['snatch'] + data['deadlift'] + data['backsq']

    data = data[['total_lift',
    'age',
    'height',
    'weight',
    'eat_I eat 1-3 full cheat meals per week',
    "eat_I eat quality foods but don't measure the amount",
    'eat_I eat strict Paleo',
    'eat_Decline to answer',
    'eat_I eat whatever is convenient',
    'eat_I weigh and measure my food',
    'background_I played college sports',
    'background_I played youth or high school level sports',
    'background_I have no athletic background besides CrossFit',
    'background_Decline to answer',
    'background_I regularly play recreational sports',
    'background_I played professional sports',
    'experience_I train other people',
    'experience_I have had a life changing experience due to CrossFit',
    'experience_I began CrossFit by trying it alone (without a coach)',
    'experience_I have completed the CrossFit Level 1 certificate course',
    'experience_Decline to answer',
    'experience_I began CrossFit with a coach (e.g. at an affiliate)',
    'experience_I have attended one or more specialty courses',
    'schedule_I do multiple workouts in a day 1x a week',
    'schedule_I do multiple workouts in a day 3+ times a week',
    'schedule_I do multiple workouts in a day 2x a week',
    'schedule_I typically rest fewer than 4 days per month',
    'schedule_I strictly schedule my rest days',
    'schedule_I typically rest 4 or more days per month',
    'schedule_Decline to answer',
    'schedule_I usually only do 1 workout a day',
    'howlong_4+ years',
    'howlong_Less than 6 months',
    'howlong_6-12 months',
    'howlong_2-4 years',
    'howlong_Decline to answer',
    'howlong_1-2 years',
    'le_region',
    'le_gender'
    ]]

    train_df, test_df = train_test_split(data,test_size=0.2,random_state=42)

    # output paths are mounted as folder, therefore, we are adding a filename to the path
    train_df.to_csv(os.path.join(args.train_data, "data.csv"), index=False)

    test_df.to_csv(os.path.join(args.test_data, "data.csv"), index=False)
    return

if __name__ == "__main__":
    main()

Overwriting ./components/data_prep/data_prep.py


In [9]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

data_prep_component = command(
    name="data_prep",
    display_name="Data preparation for training",
    description="reads a .xl input, split the input to train and test",
    inputs={
        "data": Input(type="uri_folder"),
        "test_train_ratio": Input(type="number"),
    },
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount"),
        test_data=Output(type="uri_folder", mode="rw_mount"),
    ),
    # The source folder of the component
    code=data_prep_src_dir,
    command="""python data_prep.py \
            --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} \
            --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
)

In [10]:
# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component.component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

Uploading data_prep (0.01 MBs): 100%|██████████| 9618/9618 [00:00<00:00, 135281.98it/s]




Component data_prep with Version 2023-11-01-19-56-59-5969920 is registered


**Create a versioning component**

In [11]:
# Create another component to version the data to v2
# This is similar to our data_prep component but also includes
# the transformations and feature engineering applied from assignment 1

import os

data_version_src_dir = "./components/data_version"
os.makedirs(data_version_src_dir, exist_ok=True)

In [12]:
%%writefile {data_version_src_dir}/data_version.py
import os
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import logging
import mlflow

def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.2)
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    args = parser.parse_args()

    # Start Logging
    mlflow.start_run()

    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    print("input data:", args.data)
    
    #df = pd.read_csv(args.data, header=1, index_col=0, engine='python', encoding='utf-8')
    df = pd.read_csv(args.data)
    # Retain only the first 423006 rows and 27 columns (original dataset)
    #data = df.iloc[:423006, :27]
    data = df
    # Remove not relevant columns
    data = data.dropna(subset=['region','age','weight','height','howlong','gender','eat', \
                           'train','background','experience','schedule', \
                           'deadlift','candj','snatch','backsq'])
    data = data.drop(columns=['affiliate','team','name','athlete_id','fran','helen','grace',\
                            'filthy50','fgonebad','run400','run5k','pullups','train'])

    data = data[data['weight'] < 1500]
    data = data[data['gender'] != '--']
    data = data[data['age'] >= 18]
    data = data[(data['height'] < 96) & (data['height'] > 48)]

    data = data[(data['deadlift'] > 0) & (data['deadlift'] <= 1105)|((data['gender'] == 'Female') \
                & (data['deadlift'] <= 636))]

    data = data[(data['candj'] > 0) & (data['candj'] <= 395)]
    data = data[(data['snatch'] > 0) & (data['snatch'] <= 496)]
    data = data[(data['backsq'] > 0) & (data['backsq'] <= 1069)]

    #Dummy encode eat column
    unique_outcomes = set()
    for response in data['eat']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"eat_{outcome}"
        data[column_name] = data['eat'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'eat' column if needed
    data = data.drop(columns=['eat','eat_'])

    #binary encode background column
    unique_outcomes = set()
    for response in data['background']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"background_{outcome}"
        data[column_name] = data['background'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'background' column if needed
    data = data.drop(columns=['background','background_'])

    #binary encode experience column
    unique_outcomes = set()
    for response in data['experience']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"experience_{outcome}"
        data[column_name] = data['experience'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'background' column if needed
    data = data.drop(columns=['experience','experience_'])

    #binary encode schedule column
    unique_outcomes = set()
    for response in data['schedule']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"schedule_{outcome}"
        data[column_name] = data['schedule'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'background' column if needed
    data = data.drop(columns=['schedule','schedule_'])

    #binary encode howlong column
    unique_outcomes = set()
    for response in data['howlong']:
        if '|' in response:
            outcomes = [outcome.strip() for outcome in response.split('|')]
            unique_outcomes.update(outcomes)
        else:
            unique_outcomes.add(response)

    #Create columns for each unique outcome and dummy encode
    for outcome in unique_outcomes:
        column_name = f"howlong_{outcome}"
        data[column_name] = data['howlong'].apply(lambda x: 1 if str(outcome) in str(x) else 0)

    #Drop the original 'background' column if needed
    data = data.drop(columns=['howlong','howlong_'])

    # Creating a region dictionary for maping
    region_mapping = {
        # Region 1 Group
        'Africa': 'Region_1',
        'Asia': 'Region_1',
        'Australia': 'Region_1',
        'Latin America': 'Region_1',
        
        # Region 2 Group
        'Canada East': 'Region_2',
        'Canada West': 'Region_2',

        # Region 3 Group
        'Northern California': 'Region_3',
        'Southern California': 'Region_3',
        
        # Region 4 Group
        'Central East': 'Region_4',

        # Region 5 Group
        'Mid Atlantic': 'Region_5',

        # Region 6 Group
        'North Central': 'Region_6',
        'South Central': 'Region_6',

        # Region 7 Group
        'North East': 'Region_7',
        'North West': 'Region_7',

        # Region 8 Group  
        'South East': 'Region_8',
        'South West': 'Region_8',
        
        # Region 9 Group
        'Europe': 'Region_9'
    }

    # Apply the mapping
    data['region'] = data['region'].map(region_mapping)

    #encode region and gender
    le=LabelEncoder()
    data["le_region"] = le.fit_transform(data['region'])
    data["le_gender"] = le.fit_transform(data['gender'])

    #Lets get the crosswalk for region and gender
    data.groupby(['region', 'le_region']).size().reset_index(name='counts')

    data.groupby(['gender', 'le_gender']).size().reset_index(name='counts')

    #drop unencoded region/gender columns
    data = data.drop(columns=['region','gender'])
    #calculate our total_lift target value using clean and jerk, snatch,deadlift and backsquat values
    data['total_lift'] = data['candj'] + data['snatch'] + data['deadlift'] + data['backsq']
    
    # Apply some transformations and feature engineering
    # Adding a new domain-specific feature BMI, must round then convert inches to meters and pounds to killograms.
    data['BMI'] = round((data['weight'] * 0.453592) / (data['height']*0.0254)**2,1)

    # Binning age, height, and weight, region, BMI index
    age_bins = [0, 20, 30, 40, 50, 60, 100]
    age_labels = ['0-20', '20-30', '30-40', '40-50', '50-60', '60+']  # Adjust labels as needed

    # height_bins = [0, 60, 65, 70, 75, 80, 100]
    # height_labels = ['0-60', '60-65', '65-70', '70-75', '75-80', '80+']

    # weight_bins = [0, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 750]
    # weight_labels = ['0-100', '100-125', '125-150', '150-175', '175-200', '200-225', '225-250', '250-275', '275-300',
    #                 '300-325', '325-350', '350+']

    region_bins = [0,2,5,9]
    region_labels = ['0-2','2-5','6+']

    BMI_bins = [0,16,20,25,30,35,39,70]
    BMI_labels = ['0-16','16-20','20-25','25-30','30-35','35-39','39+']

    data['binned_age'] = pd.cut(data['age'], age_bins, labels=age_labels)
    #data['binned_height'] = pd.cut(data['height'], height_bins, labels=height_labels)
    #data['binned_weight'] = pd.cut(data['weight'], weight_bins, labels=weight_labels)
    data['binned_region'] = pd.cut(data['le_region'], region_bins, labels=region_labels)
    data['binned_BMI'] = pd.cut(data['BMI'], BMI_bins, labels=BMI_labels)

    #encode new binned variables
    le=LabelEncoder()
    data["le_binned_age"] = le.fit_transform(data['binned_age'])
    #data["le_binned_height"] = le.fit_transform(data['binned_height'])
    #data["le_binned_weight"] = le.fit_transform(data['binned_weight'])
    data["le_binned_region"] = le.fit_transform(data['binned_region'])
    data["le_binned_BMI"] = le.fit_transform(data['binned_BMI'])

    # Multiply height and weight together to capture some interaction effect
    data["height_times_weight"] = data["height"] * data["weight"]

    # Square age
    data["age_squared"] = data["age"] * data["age"]

    # Square weight
    data["weight_squared"] = data["weight"] * data["weight"]

    # Square BMI
    data['BMI_squared'] = data['BMI'] * data['BMI'] 

    # Retain only certain columns for train/test, these are the same used in assignment 1
    data = data[['total_lift',
    'age_squared',
    'BMI_squared',
    'eat_I eat 1-3 full cheat meals per week',
    "eat_I eat quality foods but don't measure the amount",
    'eat_I eat strict Paleo',
    'eat_Decline to answer',
    'eat_I eat whatever is convenient',
    'eat_I weigh and measure my food',
    'background_I played college sports',
    'background_I played youth or high school level sports',
    'background_I have no athletic background besides CrossFit',
    'background_Decline to answer',
    'background_I regularly play recreational sports',
    'background_I played professional sports',
    'experience_I train other people',
    'experience_I have had a life changing experience due to CrossFit',
    'experience_I began CrossFit by trying it alone (without a coach)',
    'experience_I have completed the CrossFit Level 1 certificate course',
    'experience_Decline to answer',
    'experience_I began CrossFit with a coach (e.g. at an affiliate)',
    'experience_I have attended one or more specialty courses',
    'schedule_I do multiple workouts in a day 1x a week',
    'schedule_I do multiple workouts in a day 3+ times a week',
    'schedule_I do multiple workouts in a day 2x a week',
    'schedule_I typically rest fewer than 4 days per month',
    'schedule_I strictly schedule my rest days',
    'schedule_I typically rest 4 or more days per month',
    'schedule_Decline to answer',
    'schedule_I usually only do 1 workout a day',
    'howlong_4+ years',
    'howlong_Less than 6 months',
    'howlong_6-12 months',
    'howlong_2-4 years',
    'howlong_Decline to answer',
    'howlong_1-2 years',
    'le_binned_region',
    'le_gender'
    ]]

    train_df, test_df = train_test_split(data,test_size=0.2,random_state=42)

    # output paths are mounted as folder, therefore, we are adding a filename to the path
    train_df.to_csv(os.path.join(args.train_data, "data.csv"), index=False)

    test_df.to_csv(os.path.join(args.test_data, "data.csv"), index=False)
    return

if __name__ == "__main__":
    main()

Overwriting ./components/data_version/data_version.py


In [13]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

data_version_component = command(
    name="data_version",
    display_name="Data versioning to v2 for training",
    description="reads a .xl input, split the input to train and test",
    inputs={
        "data": Input(type="uri_folder"),
        "test_train_ratio": Input(type="number"),
    },
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount"),
        test_data=Output(type="uri_folder", mode="rw_mount"),
    ),
    # The source folder of the component
    code=data_version_src_dir,
    command="""python data_version.py \
            --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} \
            --train_data ${{outputs.train_data}} --test_data ${{outputs.test_data}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
)

In [14]:
# Now we register the component to the workspace
data_version_component = ml_client.create_or_update(data_version_component.component)

# Create (register) the component in your workspace
print(
    f"Component {data_version_component.name} with Version {data_version_component.version} is registered"
)

Component data_version with Version 2023-11-01-19-57-01-3445267 is registered


**Create Model Training Components**

**First One Will Mimic the Regression Model from Assignment 1**

In [15]:
import os

train_src_dir = "./components/train"
os.makedirs(train_src_dir, exist_ok=True)

In [16]:
%%writefile {train_src_dir}/train.py
import argparse
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.metrics import classification_report
import os
import pandas as pd
import mlflow
#import statsmodels.api as sm
#import statsmodels.stats.diagnostic as smd
#import statsmodels.stats.stattools as sms
#from statsmodels.compat import lzip
#from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression

def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])


# Start Logging
mlflow.start_run()

# enable autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)


def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    #parser.add_argument("--n_estimators", required=False, default=100, type=int)
    #parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name1", type=str, help="model name")
    parser.add_argument("--model1", type=str, help="path to model file")
    args = parser.parse_args()

    # paths are mounted as folder, therefore, we are selecting the file from folder
    train_data = pd.read_csv(select_first_file(args.train_data))

    # Extracting the label column
    y_train = train_data.pop("total_lift")

    # convert the dataframe values to array
    X_train = train_data.values

    # paths are mounted as folder, therefore, we are selecting the file from folder
    test_data = pd.read_csv(select_first_file(args.test_data))

    # Extracting the label column
    y_test = test_data.pop("total_lift")

    # convert the dataframe values to array
    X_test = test_data.values
    
    # Initialize the Linear Regression model
    model_v1 = LinearRegression() 
    # Fit the model
    model_v1.fit(X_train, y_train) 
    # Predict the training data
    y_pred = model_v1.predict(X_train) 
    # Calculate Mean Squared Error
    mse_v1 = mean_squared_error(y_train, y_pred, squared=False)

    print(f"Training with data of shape {X_train.shape}")

    #X_train = sm.add_constant(X_train)

    #model = sm.OLS(y_train,X_train)
    #results = model.fit(cov_type="HC0")
    #print(results.summary())

    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=model_v1,
        registered_model_name=args.registered_model_name1,
        artifact_path="model_v1",
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=model_v1,
        path=os.path.join(args.model1, "trained_model1"),
    )

    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()

Overwriting ./components/train/train.py


In [17]:
%%writefile {train_src_dir}/train.yml
# <component>
name: train_model
display_name: Train Model
# version: 1 # Not specifying a version will automatically update the version
type: command
inputs:
  train_data: 
    type: uri_folder
  test_data: 
    type: uri_folder    
  registered_model_name:
    type: string
outputs:
  model:
    type: uri_folder
code: .
environment:
  # for this step, we'll use an AzureML curate environment
  azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1
command: >
  python train.py 
  --train_data ${{inputs.train_data}} 
  --test_data ${{inputs.test_data}} 
  --registered_model_name ${{inputs.registered_model_name}} 
  --model ${{outputs.model}}
# </component>


Overwriting ./components/train/train.yml


In [18]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_component = load_component(source=os.path.join(train_src_dir, "train.yml"))

# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component)

# Create (register) the component in your workspace
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)

Component train_model with Version 2023-11-01-19-57-03-8244820 is registered


**Model v2 Component**

**New Model: Gradient Boost Regressor**

In [19]:
import os

trainv2_src_dir = "./components/trainv2"
os.makedirs(trainv2_src_dir, exist_ok=True)

In [20]:
%%writefile {trainv2_src_dir}/trainv2.py
from sklearn.ensemble import GradientBoostingRegressor
import argparse
import os
import pandas as pd
import mlflow
from sklearn.metrics import mean_squared_error

def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])


# Start Logging
mlflow.start_run()

# enable autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    parser.add_argument("--registered_model_name2", type=str, help="model name")
    parser.add_argument("--model2", type=str, help="path to model file")
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    args = parser.parse_args()

    # paths are mounted as folder, therefore, we are selecting the file from folder
    train_data = pd.read_csv(select_first_file(args.train_data))

    # Extracting the label column
    y_train = train_data.pop("total_lift")

    # convert the dataframe values to array
    X_train = train_data.values

    # paths are mounted as folder, therefore, we are selecting the file from folder
    test_data = pd.read_csv(select_first_file(args.test_data))

    # Extracting the label column
    y_test = test_data.pop("total_lift")

    # convert the dataframe values to array
    X_test = test_data.values

    # Initialize the Gradient Boosting Regressor model
    model_v2 = GradientBoostingRegressor(n_estimators=args.n_estimators, learning_rate=args.learning_rate)
    # Fit the model
    model_v2.fit(X_train, y_train)
    # Predict the training data
    y_pred = model_v2.predict(X_train)
    # Calculate Mean Squared Error
    mse_v2 = mean_squared_error(y_train, y_pred, squared=False)

    print(f"Training with data of shape {X_train.shape}")

    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=model_v2,
        registered_model_name=args.registered_model_name2,
        artifact_path="model_v2",
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=model_v2,
        path=os.path.join(args.model2, "trained_model2"),
    )

    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()


Overwriting ./components/trainv2/trainv2.py


In [21]:
%%writefile {trainv2_src_dir}/trainv2.yml
# <component>
name: train_modelv2
display_name: Train Model v2
# version: 1 # Not specifying a version will automatically update the version
type: command
inputs:
  train_data: 
    type: uri_folder
  test_data: 
    type: uri_folder    
  registered_model_name:
    type: string
outputs:
  model:
    type: uri_folder
code: .
environment:
  # for this step, we'll use an AzureML curate environment
  azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1
command: >
  python trainv2.py 
  --train_data ${{inputs.train_data}} 
  --test_data ${{inputs.test_data}} 
  --registered_model_name ${{inputs.registered_model_name}} 
  --model ${{outputs.model}}
# </component>


Overwriting ./components/trainv2/trainv2.yml


In [22]:
# Loading the component from the yml file
train_component2 = load_component(source=os.path.join(trainv2_src_dir, "trainv2.yml"))

# Now we register the component to the workspace
train_component2 = ml_client.create_or_update(train_component2)

# Create (register) the component in your workspace
print(
    f"Component {train_component2.name} with Version {train_component2.version} is registered"
)

Component train_modelv2 with Version 2023-11-01-19-57-05-9035667 is registered


**Model v3: Gradient Boost Regressor w/ Altered Hyperparameters**

In [23]:
import os

trainv3_src_dir = "./components/trainv3"
os.makedirs(trainv3_src_dir, exist_ok=True)

In [24]:
%%writefile {trainv3_src_dir}/trainv3.py
from sklearn.ensemble import GradientBoostingRegressor
import argparse
import os
import pandas as pd
import mlflow
from sklearn.metrics import mean_squared_error

def select_first_file(path):
    """Selects first file in folder, use under assumption there is only one file in folder
    Args:
        path (str): path to directory or file to choose
    Returns:
        str: full path of selected file
    """
    files = os.listdir(path)
    return os.path.join(path, files[0])


# Start Logging
mlflow.start_run()

# enable autologging
mlflow.sklearn.autolog()

os.makedirs("./outputs", exist_ok=True)

def main():
    """Main function of the script."""

    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, help="path to train data")
    parser.add_argument("--test_data", type=str, help="path to test data")
    parser.add_argument("--registered_model_name3", type=str, help="model name")
    parser.add_argument("--model3", type=str, help="path to model file")
    parser.add_argument("--n_estimators", required=False, default=200, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.05, type=float)
    args = parser.parse_args()

    # paths are mounted as folder, therefore, we are selecting the file from folder
    train_data = pd.read_csv(select_first_file(args.train_data))

    # Extracting the label column
    y_train = train_data.pop("total_lift")

    # convert the dataframe values to array
    X_train = train_data.values

    # paths are mounted as folder, therefore, we are selecting the file from folder
    test_data = pd.read_csv(select_first_file(args.test_data))

    # Extracting the label column
    y_test = test_data.pop("total_lift")

    # convert the dataframe values to array
    X_test = test_data.values

    # Initialize the Gradient Boosting Regressor model
    model_v3 = GradientBoostingRegressor(n_estimators=args.n_estimators, learning_rate=args.learning_rate)
    # Fit the model
    model_v3.fit(X_train, y_train)
    # Predict the training data
    y_pred = model_v3.predict(X_train)
    # Calculate Mean Squared Error
    mse_v3 = mean_squared_error(y_train, y_pred, squared=False)

    print(f"Training with data of shape {X_train.shape}")

    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=model_v3,
        registered_model_name=args.registered_model_name3,
        artifact_path="model_v3",
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=model_v3,
        path=os.path.join(args.model3, "trained_model3"),
    )

    # Stop Logging
    mlflow.end_run()


if __name__ == "__main__":
    main()


Overwriting ./components/trainv3/trainv3.py


In [25]:
%%writefile {trainv3_src_dir}/trainv3.yml
# <component>
name: train_modelv3
display_name: Train Model v3
# version: 1 # Not specifying a version will automatically update the version
type: command
inputs:
  train_data: 
    type: uri_folder
  test_data: 
    type: uri_folder    
  registered_model_name:
    type: string
outputs:
  model:
    type: uri_folder
code: .
environment:
  # for this step, we'll use an AzureML curate environment
  azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1
command: >
  python trainv3.py 
  --train_data ${{inputs.train_data}} 
  --test_data ${{inputs.test_data}} 
  --registered_model_name ${{inputs.registered_model_name}} 
  --model ${{outputs.model}}
# </component>


Overwriting ./components/trainv3/trainv3.yml


In [26]:
# Loading the component from the yml file
train_component3 = load_component(source=os.path.join(trainv3_src_dir, "trainv3.yml"))

# Now we register the component to the workspace
train_component3 = ml_client.create_or_update(train_component3)

# Create (register) the component in your workspace
print(
    f"Component {train_component3.name} with Version {train_component3.version} is registered"
)

Component train_modelv3 with Version 2023-11-01-19-57-07-8823370 is registered


**Pipeline Creation**

**Experiment 1**


In [27]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute=cpu_compute_target
    if (cpu_cluster)
    else "serverless",  # "serverless" value runs pipeline on serverless compute
    description="Experiment 1 with Cleaned Data and Original Linear Regression Model",
)
def pipeline(
    pipeline_job_data_input,
    pipeline_job_test_train_ratio,
    pipeline_job_learning_rate,
    pipeline_job_registered_model_name,
):
    # using data_prep_function like a python call with its own inputs
    data_prep_job = data_prep_component(
        data=pipeline_job_data_input,
        test_train_ratio=pipeline_job_test_train_ratio,
    )

    # using train_func like a python call with its own inputs
    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        registered_model_name=pipeline_job_registered_model_name,
    )

    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_train_data": data_prep_job.outputs.train_data,
        "pipeline_job_test_data": data_prep_job.outputs.test_data,
    }

In [28]:
registered_model_name = "model1"

# Let's instantiate the pipeline with the parameters of our choice
pipeline = pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=data_asset.path),
    pipeline_job_test_train_ratio=0.2,
    pipeline_job_registered_model_name=registered_model_name,
)

In [29]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="Experiment 1",
)
ml_client.jobs.stream(pipeline_job.name)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Request time out. Ingestion may be bac

RunId: eager_wire_60q4zhc5l3
Web View: https://ml.azure.com/runs/eager_wire_60q4zhc5l3?wsid=/subscriptions/5405ca59-d0a7-4e18-af16-b7f21ebcb1b0/resourcegroups/mlops_group6/workspaces/mlops_assignment2

Streaming logs/azureml/executionlogs.txt

[2023-11-01 19:57:57Z] Submitting 1 runs, first five are: ac18da2f:141ad396-67cb-4b84-bebf-0c5d6497e97f
[2023-11-01 19:59:23Z] Completing processing run id 141ad396-67cb-4b84-bebf-0c5d6497e97f.
[2023-11-01 19:59:24Z] Submitting 1 runs, first five are: 7e144ff0:40f216ae-db5d-4f4a-b9be-182fd37bf75a
[2023-11-01 20:01:30Z] Completing processing run id 40f216ae-db5d-4f4a-b9be-182fd37bf75a.

Execution Summary
RunId: eager_wire_60q4zhc5l3
Web View: https://ml.azure.com/runs/eager_wire_60q4zhc5l3?wsid=/subscriptions/5405ca59-d0a7-4e18-af16-b7f21ebcb1b0/resourcegroups/mlops_group6/workspaces/mlops_assignment2



**Creating new versions of the pipeline to run different experiments**

**Experiment 2: Data Version 2**

In [44]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output

@dsl.pipeline(
    compute=cpu_compute_target
    if (cpu_cluster)
    else "serverless",  # "serverless" value runs pipeline on serverless compute
    description="Experiment V2 with DataV2",
)
def pipeline(
    pipeline_job_data_input,
    pipeline_job_test_train_ratio,
    pipeline_job_learning_rate,
    pipeline_job_registered_model_name,
):
    # using data_version_function like a python call with its own inputs
    data_version_job = data_version_component(
        data=pipeline_job_data_input,
        test_train_ratio=pipeline_job_test_train_ratio,
    )

    # using train_func like a python call with its own inputs
    train_job = train_component(
        train_data=data_version_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_version_job.outputs.test_data,  # note: using outputs from previous step
        registered_model_name=pipeline_job_registered_model_name,
    )

    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_train_data": data_version_job.outputs.train_data,
        "pipeline_job_test_data": data_version_job.outputs.test_data,
    }

In [45]:
registered_model_name = "model2"

# Let's instantiate the pipeline with the parameters of our choice
pipeline = pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=data_asset.path),
    pipeline_job_test_train_ratio=0.2,
    pipeline_job_registered_model_name=registered_model_name,
)

In [46]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="Experiment 2 With Data V2",
)
ml_client.jobs.stream(pipeline_job.name)

RunId: jovial_airport_3n8g4nzj93
Web View: https://ml.azure.com/runs/jovial_airport_3n8g4nzj93?wsid=/subscriptions/5405ca59-d0a7-4e18-af16-b7f21ebcb1b0/resourcegroups/mlops_group6/workspaces/mlops_assignment2

Streaming logs/azureml/executionlogs.txt

[2023-11-01 17:43:24Z] Completing processing run id f4486d2e-e620-40d9-a236-43f7d76fa30b.
[2023-11-01 17:43:25Z] Submitting 1 runs, first five are: 770a2ccb:c1fc51fb-c1c5-4f3d-9036-31f39e57e2eb
[2023-11-01 17:44:22Z] Completing processing run id c1fc51fb-c1c5-4f3d-9036-31f39e57e2eb.

Execution Summary
RunId: jovial_airport_3n8g4nzj93
Web View: https://ml.azure.com/runs/jovial_airport_3n8g4nzj93?wsid=/subscriptions/5405ca59-d0a7-4e18-af16-b7f21ebcb1b0/resourcegroups/mlops_group6/workspaces/mlops_assignment2



**Experiment V3 with Data Version V2 and Model V2**

In [47]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output

@dsl.pipeline(
    compute=cpu_compute_target
    if (cpu_cluster)
    else "serverless",  # "serverless" value runs pipeline on serverless compute
    description="Experiment V3 with DataV2 and Model V2",
)
def pipeline(
    pipeline_job_data_input,
    pipeline_job_test_train_ratio,
    pipeline_job_learning_rate,
    pipeline_job_registered_model_name,
):
    # using data_version_function like a python call with its own inputs
    data_version_job = data_version_component(
        data=pipeline_job_data_input,
        test_train_ratio=pipeline_job_test_train_ratio,
    )

    # using train_func like a python call with its own inputs
    train_job = train_component2(
        train_data=data_version_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_version_job.outputs.test_data,  # note: using outputs from previous step
        registered_model_name=pipeline_job_registered_model_name,
    )

    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_train_data": data_version_job.outputs.train_data,
        "pipeline_job_test_data": data_version_job.outputs.test_data,
    }

In [48]:
registered_model_name = "model3"

# Let's instantiate the pipeline with the parameters of our choice
pipeline = pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=data_asset.path),
    pipeline_job_test_train_ratio=0.2,
    pipeline_job_registered_model_name=registered_model_name,
)

In [49]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="Experiment 3 With Data V2 and Model V2",
)
ml_client.jobs.stream(pipeline_job.name)

RunId: quirky_cart_x4m2j217lp
Web View: https://ml.azure.com/runs/quirky_cart_x4m2j217lp?wsid=/subscriptions/5405ca59-d0a7-4e18-af16-b7f21ebcb1b0/resourcegroups/mlops_group6/workspaces/mlops_assignment2
[2023-11-01 17:45:29Z] Completing processing run id f6f46854-717b-486f-bbec-a94af72747e1.

Execution Summary
RunId: quirky_cart_x4m2j217lp
Web View: https://ml.azure.com/runs/quirky_cart_x4m2j217lp?wsid=/subscriptions/5405ca59-d0a7-4e18-af16-b7f21ebcb1b0/resourcegroups/mlops_group6/workspaces/mlops_assignment2



**Experiment V4 With Data Version V2 and Model V2 with different hyperparameters**

In [50]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output

@dsl.pipeline(
    compute=cpu_compute_target
    if (cpu_cluster)
    else "serverless",  # "serverless" value runs pipeline on serverless compute
    description="Experiment V4 with DataV2, Model V2, and Altered Hyperparameters",
)
def pipeline(
    pipeline_job_data_input,
    pipeline_job_test_train_ratio,
    pipeline_job_learning_rate,
    pipeline_job_registered_model_name,
):
    # using data_version_function like a python call with its own inputs
    data_version_job = data_version_component(
        data=pipeline_job_data_input,
        test_train_ratio=pipeline_job_test_train_ratio,
    )

    # using train_func like a python call with its own inputs
    train_job = train_component3(
        train_data=data_version_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_version_job.outputs.test_data,  # note: using outputs from previous step
        registered_model_name=pipeline_job_registered_model_name,
    )

    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_train_data": data_version_job.outputs.train_data,
        "pipeline_job_test_data": data_version_job.outputs.test_data,
    }

In [51]:
registered_model_name = "model4"

# Let's instantiate the pipeline with the parameters of our choice
pipeline = pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=data_asset.path),
    pipeline_job_test_train_ratio=0.2,
    pipeline_job_registered_model_name=registered_model_name,
)

In [52]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="Experiment 4 With Data V2, Model V2, and Altered Hyperparameters",
)
ml_client.jobs.stream(pipeline_job.name)

RunId: coral_nail_qm23phc3hq
Web View: https://ml.azure.com/runs/coral_nail_qm23phc3hq?wsid=/subscriptions/5405ca59-d0a7-4e18-af16-b7f21ebcb1b0/resourcegroups/mlops_group6/workspaces/mlops_assignment2

Streaming logs/azureml/executionlogs.txt

[2023-11-01 17:45:38Z] Completing processing run id 36da371e-fa9f-41b1-8236-62774d308091.
[2023-11-01 17:45:39Z] Submitting 1 runs, first five are: 9491b583:79c5f2f5-0d14-48a9-a24c-fd263276165e
[2023-11-01 17:46:33Z] Completing processing run id 79c5f2f5-0d14-48a9-a24c-fd263276165e.

Execution Summary
RunId: coral_nail_qm23phc3hq
Web View: https://ml.azure.com/runs/coral_nail_qm23phc3hq?wsid=/subscriptions/5405ca59-d0a7-4e18-af16-b7f21ebcb1b0/resourcegroups/mlops_group6/workspaces/mlops_assignment2

